# Colonoscopy Polyp Detection w/Faster R-CNN and Lightning

In [10]:
!pwd

/notebooks/LightningFasterRCNN


## imports and setup

In [1]:
# %load_ext autoreload
# %autoreload 2

import os
from pathlib import Path
import torch
from src.LightningModuleFCOS import LightningModuleFCOS as pl_module
from src.LightningModuleFCOS import MetricsCallback as metrics_callback


In [2]:
CONFIG = dict (
    project = "hlc-polyp-detection",
    architecture = "fcos_resnet50_fpn",
    dataset_id = "hlc-custom-polyp-detection",
    infra = "osx",
    num_classes = 2,
    max_epochs = 100,
    lr=0.01,
    min_lr=0.0000001,
    epochs=15,
    batch_size=4,
    nesterov=True,
    momentum=0.9,
    weight_decay=0.0005,
    clip_limit=0.25,
    difference=False,
    name="fancy_walrus"
)

ROOT_DIR = os.path.abspath("./")
DATA_DIR = os.path.join(ROOT_DIR, "data")
MODEL_DIR = os.path.join(ROOT_DIR, "models/fcos_resnet50_fpn")
LOG_DIR = os.path.join(ROOT_DIR, "log")

NUM_CLASSES = 2

BATCH_SIZE = 4
INPUT_SIZE = 1024
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 4

wandb_key = Path(os.path.join(ROOT_DIR, "wandb.txt")).read_text().strip()
os.environ["WANDB_API_KEY"] = wandb_key
os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(ROOT_DIR, "LightningFastRCNN-polyps.ipynb")

In [3]:
if CONFIG["infra"] == "paperspace":
    # !pip install -r alubumentations pytorch-lightning wandb --upgrade
    import wandb
    !nvidia-smi
else:
    import wandb

In [4]:
from contextlib import contextmanager

@contextmanager
def wandb_context(configuration=CONFIG):
    run = wandb.init(reinit=True, config=configuration, project=CONFIG['project'])
    try:
        yield run
    finally:
        wandb.finish()

In [5]:
from src.PolypsPLDataModule import PolypsPLDataModule

polyp_dm = PolypsPLDataModule(data_dir='./data', batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping

def train_model(model, run_name, dm=None, run=None):
    wandb_logger = None
    if run is not None:
        run.config["train_run_name"] = run_name

        from pytorch_lightning.loggers import WandbLogger
        wandb_logger = WandbLogger()
    
    chkpt = ModelCheckpoint(
        dirpath=os.path.join(MODEL_DIR, "chkpts"),
        filename=f"{CONFIG['name']}-chkpt-{run_name}",
        monitor="val_recall",
        mode="max")
    
    lrnrate = LearningRateMonitor(logging_interval="step", log_momentum=True)

    earlystop = EarlyStopping(
        monitor="val_recall",
        patience=50,
        verbose=True,
        mode="max")
    
    trainer = Trainer(
        accelerator="gpu" if torch.cuda.is_available() else "cpu", 
        logger=wandb_logger,
        callbacks=[chkpt, metrics_callback(), lrnrate, earlystop],
        log_every_n_steps=1,
        max_epochs=CONFIG["max_epochs"])
    
    trainer.fit(
        model,
        datamodule=dm)
    
    return pl_module.load_from_checkpoint(chkpt.best_model_path)

In [7]:
%load_ext autoreload
%autoreload 2

In [11]:
# this should have been:
# with wandb_context(CONFIG) as run:
#     model = LightningFasterModule()
from src.LightningModuleFCOS import LightningModuleFCOS as pl_module
with wandb_context() as run:
    model = pl_module(CONFIG)
    if run is not None:
        # watch the hyperparameters and gradients of the model 
        run.watch(model)
        model.full_train()
        model = train_model(model, run_name="full", dm=polyp_dm, run=run)
    



/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name     | Type | Params
----------------------------------
0 | detector | FCOS | 32.1 M
----------------------------------
32.1 M    Trainable params
0         Non-trainable params
32.1 M    Total params
128.258   Total estimated model params size (MB)


Epoch 00000: adjusting learning rate of group 0 to 1.0000e-02.


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: 'LightningModuleFCOS' object has no attribute 'test_metrics_dict'